In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import cv2 as cv
from path import Path
import os 
import pydicom as dicom
import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.keras import regularizers
from keras.callbacks import LearningRateScheduler
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from pydicom import dcmread

In [ ]:
train_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")

In [ ]:
train_df

In [ ]:
def load_dicom(path):
    img=dicom.dcmread(path)
    data=img.pixel_array
    data=data-np.min(data)
    if np.max(data) != 0:
        data=data/np.max(data)
    data=(data*255).astype(np.uint8)
    return data

In [ ]:
def lr_schedule(epoch):
    lr = 0.1
    if epoch > 30:
        lr = 0.1
    if epoch > 70:
        lr = 0.01
    return lr

In [ ]:
def listdirs(folder):
    return [d for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))]

In [ ]:
train_dir='../input/rsna-2022-cervical-spine-fracture-detection/train_images'

In [ ]:
from pydicom.data import get_testdata_files
trainset=[]
trainlabel=[]
trainidt=[]
limit = 251
for i in tqdm(range(len(train_df))): 
    idt=train_df.loc[i,'StudyInstanceUID']

    path=os.path.join(train_dir,idt)   
    
    for im in os.listdir(path):
        
        
        dc = dicom.read_file(os.path.join(path,im))
        if dc.file_meta.TransferSyntaxUID.name =='JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])':
            continue
        
        img=load_dicom(os.path.join(path,im)) 

        img=cv.resize(img,(64,64)) 
        image=img_to_array(img)
        image=image/255.0

        trainset+=[image]
        cur_label=[]
        cur_label.append(train_df.loc[i, 'patient_overall'])
        cur_label.append(train_df.loc[i,'C1'])
        cur_label.append(train_df.loc[i,'C2'])
        cur_label.append(train_df.loc[i,'C3'])
        cur_label.append(train_df.loc[i,'C4'])
        cur_label.append(train_df.loc[i,'C5'])
        cur_label.append(train_df.loc[i,'C6'])
        cur_label.append(train_df.loc[i,'C7'])
        trainlabel+=[cur_label]
        trainidt+=[idt]

    i+=1
    if i==limit:
        break

In [ ]:
y=np.array(trainlabel)
Y_train=y
X_train=np.array(trainset)

In [ ]:
test_df = ['1.2.826.0.1.3680043.22327', '1.2.826.0.1.3680043.25399', '1.2.826.0.1.3680043.5876']

In [ ]:
test_dir='../input/rsna-2022-cervical-spine-fracture-detection/test_images'
testset=[]
testidt=[]
for i in tqdm(range(len(test_df))):
    idt=test_df[i]
    path=os.path.join(test_dir,idt)   
    
    for im in os.listdir(path):
        dc = dicom.read_file(os.path.join(path,im))
        
        if dc.file_meta.TransferSyntaxUID.name =='JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])':
            continue
        img=load_dicom(os.path.join(path,im)) 
        img=cv.resize(img,(64,64)) 
        image=img_to_array(img)
        image=image/255.0
        testset+=[image]
        testidt+=[idt]

In [ ]:
X_test = np.array(testset)

In [ ]:
model1 = Sequential()
model1.add(Conv2D(64, (4, 4), activation='relu', input_shape=(64, 64, 1)))
model1.add(MaxPooling2D((2, 2)))
model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dense(11, activation='softmax'))

model2 = Sequential()
model2.add(Conv2D(64, (4, 4), activation='relu', input_shape=(64, 64, 1)))
model2.add(MaxPooling2D((2, 2)))
model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
model2.add(Dense(11, activation='softmax'))

model3 = Sequential()
model3.add(Conv2D(64, (4, 4), activation='relu', input_shape=(64, 64, 1)))
model3.add(Conv2D(64, (4, 4), activation='relu'))
model3.add(MaxPooling2D((2, 2)))
model3.add(Flatten())
model3.add(Dense(128, activation='relu'))
model3.add(Dense(11, activation='softmax'))

input_layer = Input(shape=(64, 64, 1))
model1_output = model1(input_layer)
model2_output = model2(input_layer)
model3_output = model3(input_layer)
merged = concatenate([model1_output, model2_output, model3_output])
ensemble_output = Dense(11, activation='softmax')(merged)
model = Model(inputs=input_layer, outputs=ensemble_output)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
hist = model.fit(X_train, Y_train, epochs=40, batch_size=64, verbose=1, callbacks=[lr_scheduler])

In [ ]:
test_loss = model.evaluate(X_train, Y_train)
total_loss = test_loss[0]
total_accuracy = test_loss[1]

In [ ]:
print('Testing Loss:', total_loss)
print('Testing Accuracy:', total_accuracy)

In [ ]:
y_pred=model.predict(X_test)
values = np.max(y_pred, axis = 1)
pred=np.argmax(y_pred,axis=1)

In [ ]:
averages = []
averages1 = []
averages2 = []
for i in range(len(y_pred[0])):
    total = 0
    for j in range(313):
        total += y_pred[j][i]
    averages1.append(total/len(y_pred))
averages.append(averages1)

In [ ]:
result = pd.DataFrame()
result[0] = pd.Series(testidt2).astype(str) + '_C' + pd.Series(range(1,9)).astype(str)
result.iloc[-1,0] = str(testid) + '_patient_overall'
result[1]=averages1

In [ ]:
result

In [ ]:
result.to_csv('submission.csv', index=False, float_format='%.1g')